In [45]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from flask import Flask, jsonify

import pandas as pd
import numpy as np
import os
import datetime as dt

In [46]:
#################################################
# Database Setup
#################################################
engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save reference to the tables
Measurements = Base.classes.measurements
Stations = Base.classes.stations

# Create our session (link) from Python to the DB
session = Session(engine)

#################################################
# Flask Setup
#################################################
app = Flask(__name__)


In [47]:
#################################################
# Flask Routes
#################################################

@app.route("/")
def welcome():
    """List all available api routes."""
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs"
#         f"/api/v1.0/<start><br/>"
#         f"/api/v1.0/<start>/<end><br/>"
    )

@app.route("/api/v1.0/tobs")
def tobs():
    """Query for the dates and temperature observations from the last year.

      * Convert the query results to a Dictionary using `date` as the key and `tobs` as the value.

      * Return the json representation of your dictionary."""
    startDate = dt.date(2014, 8 ,15)
    # date 1 year ago from start_date
    year_ago = startDate - dt.timedelta(days=365)
    
    sel = [Measurements.date, Measurements.tobs]

    tobs_data = session.query(*sel).\
    filter(Measurements.station == Stations.station).filter(Measurements.date <= startDate).\
    filter(Measurements.date >= year_ago).all()
    
    tobs_data

    # Convert list of tuples into normal list
    all_prec = list(np.ravel(tobs_data))
    #prec_dict = {k: all_prec[0] for k in all_prec[1]}
    return jsonify(all_prec)


# Return a json list of stations from the dataset.
@app.route("/api/v1.0/stations")
def stations():
    
    sel = [Stations.name]

    tobs_data = session.query(*sel).filter(Measurements.station == Stations.station).group_by(Measurements.station).all()
    
    tobs_data

    # Convert list of tuples into normal list
    all_data = list(np.ravel(tobs_data))
    return jsonify(all_data)

@app.route("/api/v1.0/precipitation")
def precipitation():
    startDate = dt.date(2014, 8 ,15)
    # date 1 year ago from start_date
    year_ago = startDate - dt.timedelta(days=365)
    
    sel = [Measurements.date, Measurements.prcp]

    prcp_data = session.query(*sel).\
    filter(Measurements.station == Stations.station).filter(Measurements.date <= startDate).\
    filter(Measurements.date >= year_ago).all()
    
    prcp_data

    # Convert list of tuples into normal list
    all_prec = list(np.ravel(prcp_data))
    #prec_dict = {k: all_prec[0] for k in all_prec[1]}
    return jsonify(all_prec)


if __name__ == '__main__':
    app.run(debug=False) # debug=False for Jupyter notebook

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Apr/2018 13:38:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2018 13:38:24] "GET /api/v1.0/precipitation HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2018 13:38:32] "GET /api/v1.0/precipitation HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2018 13:38:39] "GET /api/v1.0/stations HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2018 13:38:45] "GET /api/v1.0/tobs HTTP/1.1" 200 -
